## 对结构化数据进行分类

### 导入TensorFlow和其他库

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.0.0-alpha0


### 使用Pandas导入数据集

In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
df = pd.read_csv(URL)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


### 将数据集拆分为训练、验证和测试集

In [3]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


### 使用tf.data创建输入管道

In [26]:
# 从Pandas的dataframe创建tf.data数据集
def df_to_dataset(df, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [5]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

### 进一步了解输入管道

In [6]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of ages:', feature_batch['age'])
    print('A batch of targets:', label_batch)

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([52 40 62 64 37], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int32)


### 演示几种特征列(feature column)

In [7]:
example_batch = next(iter(train_ds))[0]

In [8]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

#### 数值列

In [10]:
age = feature_column.numeric_column('age')
demo(age)

[[52.]
 [40.]
 [62.]
 [64.]
 [37.]]


#### 桶列（Bucketized）

In [12]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


#### 分类列

In [28]:
thal = feature_column.categorical_column_with_vocabulary_list(
        'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


#### 嵌入列

In [29]:
# 注意，传递给嵌入列的是一个分类列
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[-0.6782604  -0.511632    0.01026161 -0.4221003   0.118221   -0.6251951
  -0.16774575 -0.17677607]
 [ 0.45159644 -0.05390342  0.03806166 -0.6353068  -0.0791701  -0.27596644
   0.49114797 -0.49382535]
 [ 0.45159644 -0.05390342  0.03806166 -0.6353068  -0.0791701  -0.27596644
   0.49114797 -0.49382535]
 [ 0.45159644 -0.05390342  0.03806166 -0.6353068  -0.0791701  -0.27596644
   0.49114797 -0.49382535]
 [-0.6782604  -0.511632    0.01026161 -0.4221003   0.118221   -0.6251951
  -0.16774575 -0.17677607]]


#### 散列（hash）特征列

In [17]:
thal_hased = feature_column.categorical_column_with_hash_bucket(
    'thal', hash_bucket_size=1000)
thal_hased_one_hot = feature_column.indicator_column(thal_hased)
demo(thal_hased_one_hot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#### 交叉特征列

In [19]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### 选择合适的特征列

In [20]:
feature_columns = []

# numeric columns
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))

# bucketized columns
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator columns
thal = feature_column.categorical_column_with_vocabulary_list(
        'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embeddding columns
thal_embeddding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed columns
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

#### 创建一个特征层

In [21]:
feature_layer = layers.DenseFeatures(feature_columns)

In [22]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

### 创建、编译和训练模型

In [23]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
7/7 [==============================] - 0s 71ms/step - loss: 1.0957 - accuracy: 0.6601 - val_loss: 0.6680 - val_accuracy: 0.6735
Epoch 2/5
7/7 [==============================] - 0s 27ms/step - loss: 0.5393 - accuracy: 0.7600 - val_loss: 0.5512 - val_accuracy: 0.6939
Epoch 3/5
7/7 [==============================] - 0s 27ms/step - loss: 0.6062 - accuracy: 0.7403 - val_loss: 0.6553 - val_accuracy: 0.6735
Epoch 4/5
7/7 [==============================] - 0s 27ms/step - loss: 0.5936 - accuracy: 0.7410 - val_loss: 0.7399 - val_accuracy: 0.6735
Epoch 5/5
7/7 [==============================] - 0s 27ms/step - loss: 0.4790 - accuracy: 0.7734 - val_loss: 0.5646 - val_accuracy: 0.7143


In [24]:
loss, accuracy = model.evaluate(test_ds)
print('Accuracy:', accuracy)

2/2 [==============================] - 0s 20ms/step - loss: 0.6159 - accuracy: 0.7213
Accuracy: 0.72131145
